# Bounty Hunting Project Notebook

< Insert group member names here and change the name of the file so it has your group ID in it.>

## Set Up

First, we load in some helper files.

In [1]:
import sys
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
sys.path.append('dontlook')
from dontlook import bountyHuntData
from dontlook import bountyHuntWrapper

Next, we load in the data. You should use `train_x` and `train_y` to train your models. The second set of data (`validation_x` and `validation_y`) is for testing your models, to ensure that you aren't overfitting. It is also what will be passed to the updater in order to determine if a proposed update should be accepted and if repairs are needed. Since you have access to this data, you could overfit to it and get a bunch of updates accepted. However, a) we'll be able to tell you did this and b) your updates will fail on the holdout set that only we have access to, so doing this is not in your best interest.

In [2]:
[train_x, train_y, validation_x, validation_y] = bountyHuntData.get_data()

The model that you'll be building off of is a decision stump, i.e. a very stupid decision list with only one node. **Warning: do not rerun the next code block unless you want to completely restart building your PDL, as it will re-initialize it to just the decision stump!**

In [3]:
initial_model = DecisionTreeClassifier(max_depth = 1, random_state=0)
initial_model.fit(train_x, train_y)
f = bountyHuntWrapper.build_initial_pdl(initial_model, train_x, train_y, validation_x, validation_y)

# Bounty Hunting

Here's where the bulk of the work you'll be doing will live. Your job is to generate groups g such that there is some h that does better than the current model f on that group. Here, we generate an example group function, which identifies African American individuals.

In [4]:
# def g1(x):
#     """
#     Given an x, g should return either 0 or 1.
#     :param x: input vector
#     :return: 0 or 1; 1 if x belongs to group, 0 otherwise
#     """

#     # here is how to make a function g that returns 1 for all African American individuals
#     if x['RAC1P'] == 2:
#         return 1
#     else:
#         return 0

You might also imaging making a group function that tries to learn what regions the current algorithm performs poorly on in an adaptive way, instead of just guessing ad-hoc that it will do poorly on a particular subgroup. In order to generate such a g, you will need to generate a constructor that takes as input a current model and the training data, and outputs a function g. A template for doing this is provided below. The example version returns a very silly function g which looks at the predictions the current model makes, and returns a group function where the group it has learned is all the points that the PDL labels as a 1. It completely ignores the true labels (train_y), so is probably not a very good group function.

In [5]:
# Don't execute this cell unless you are trying to build complicated groups

# import numpy as np


# def g_(f,train_x, train_y):
#     # f is the current PDL
#     preds = train_x.apply(f.predict, axis=1)
#     xs = train_x[preds == 1]
#     ys = train_y[preds == 1]
#     dt = DecisionTreeClassifier(max_depth = 1, random_state=0)
#     dt.fit(xs, ys)
#     def g(x):
#         # g should take as input a SINGLE x and return 0 or 1 for it.
#         # if we call dt.predict on x it will break because the dimensions of x are wrong, so we have to reshape it and reshape the output.
#         # this is not particularly efficient, so if you have better ways of doing this go for it. :)
#         y = dt.predict(np.array(x).reshape(1,-1))
#         return y[0]
#     return g

# # if you wanted to build a particular g using the above, you could use the following line.
# g = g_(f, train_x, train_y)

In the following cell(s), generate group functions that you think will make improvements, and then try to run the updates as explained in the subsequent section. In the final version of your code that you turn in, the groups that you generated and their corresponding models h, and the order in which you did updates, should be obvious and re-generating your final PDL should be completely reproducible just by running the code blocks in this notebook.

In [6]:
# # Add your code defining groups (and possibly hs) here.

Once you've found a promising group g, you can run the following updater code. Here, we define two different update functions. The first, `simple_updater`, only requires that you find some model g that you think f might do poorly on. Then, it automatically trains a decision list of depth 10 on the training data restricted to your g, and it passes that model and g along to the updater.

You might want to do something a bit fancier than a decision tree to make your model, in which case you can run the second updater, which takes as input a group g and model h, and then updates f accordingly.

Every time you run the update function, it will tell you if your (g,h) passed the validation checks, i.e. if a) your group existed in the validation data and b) it made an improvement compared to f. If it did pass the validation checks, then the model f is updated to include your g and h. **Note that this means that as you run updates, it will be increasingly difficult to find groups that make improvements.**

In [7]:
def simple_updater(g,group_name = "g"):
    # if you want to change how h is trained, you can edit the below line.
    h = bountyHuntWrapper.build_model(train_x, train_y, g, dt_depth=10)
    # do not change anything beyond this point.
    if bountyHuntWrapper.run_checks(f, validation_x, validation_y, g, h, train_x=train_x, train_y=train_y):
        print("Running Update")
        bountyHuntWrapper.run_updates(f, g, h, train_x, train_y, validation_x, validation_y, group_name=group_name)
        
def rf_updater(g, group_name = "g", dt_depth = 5):
    print("building h")
    # Get indices first
    indices = train_x.apply(g, axis=1) == 1
    # Find training set
    training_xs = train_x[indices]
    training_ys = train_y[indices]

    clf = RandomForestClassifier(max_depth=dt_depth, random_state=0)  # setting random state for replicability
    clf.fit(training_xs, training_ys)
    print("finished building h")
    h = clf.predict
    # do not change anything beyond this point.
    if bountyHuntWrapper.run_checks(f, validation_x, validation_y, g, h, train_x=train_x, train_y=train_y):
        print("Running Update")
        bountyHuntWrapper.run_updates(f, g, h, train_x, train_y, validation_x, validation_y, group_name=group_name)

def updater(g, h, group_name="g"):
    # do not alter this code
    if bountyHuntWrapper.run_checks(f, validation_x, validation_y, g, h, train_x=train_x, train_y=train_y):
        print("Running Update")
        bountyHuntWrapper.run_updates(f, g, h, train_x, train_y, validation_x, validation_y, group_name=group_name)

In the below block, provide a script that builds *the entire final PDL that you come up with*. We will run this on the initial version of f (the decision stump) in order to evaluate your code. (Note: it is fine for the group functions g and the hs to be defined in above code blocks, just make sure that everything runs as you expect if you run everything from a clean kernel)

In [8]:
def g1(x):
    """
    Given an x, g should return either 0 or 1.
    :param x: input vector
    :return: 0 or 1; 1 if x belongs to group, 0 otherwise
    """

    # here is how to make a function g that returns 1 for all African American individuals
    if x['RAC1P'] == 4 or x['RAC1P'] == 5 or x['RAC1P'] == 7:
        return 1
    else:
        return 0
simple_updater(g1,group_name="g1")
# rf_updater(g1,group_name="g1")

building h
finished building h
Error of current model on proposed group: 0.5
Error of h trained on proposed group: 0.25
Group size in test set: 40
Group weight in test set: 0.0013538669825689626
Training Error of current model on proposed group: 0.36170212765957444
Training Error of h trained on proposed group: 0.05531914893617018
Group size in training set: 235
Group weight in training set: 0.0017044300675969713
Passed checks.
Running Update
updating errors
getting new errors
prob node -1
new errs [0.3452022338805213, 0.25]


In [9]:
def g2(x):
    if x['AGEP'] < 16 or x['AGEP'] > 60:
        return 1
    else:
        return 0
simple_updater(g2,group_name="g2")
# rf_updater(g2,group_name="g2")

building h
finished building h
Error of current model on proposed group: 0.451381127255309
Error of h trained on proposed group: 0.12877215391984675
Group size in test set: 12526
Group weight in test set: 0.42396344559147064
Training Error of current model on proposed group: 0.446218371362491
Training Error of h trained on proposed group: 0.12285546618799403
Group size in training set: 57938
Group weight in training set: 0.4202181670486524
Passed checks.
Running Update
updating errors
getting new errors
prob node -1
new errs [0.2084278219664918, 0.17500000000000004, 0.12877215391984675]


In [ ]:
def g3(x):
    if x['MAR'] == 1:
        return 0
    else:
        return 1
simple_updater(g3,group_name="g3")

In [ ]:
def g4(x):
    if x['SCHL'] < 16:
        return 1
    else:
        return 0
simple_updater(g4,group_name="g4")

In [ ]:
def g5(x):
    if x['MIL'] != 1 and x['MIL'] != 5 and x['SEX'] == 2:
        return 1
    else:
        return 0
simple_updater(g5,group_name="g5")

In [ ]:
def g6(x):
    if x['DIS'] == 1:
        return 1
    else:
        return 0
simple_updater(g6,group_name="g6")

In [ ]:
def g7(x):
    if x['MIL'] == 4:
        return 0
    else:
        return 1
simple_updater(g7,group_name="g7")

In [ ]:
# def g8(x):
#     if x['RAC1P'] == 2 and x['SEX'] == 2:
#         return 1
#     else:
#         return 0
    
def g8(x):
    if (x['AGEP'] < 22 or x['AGEP'] > 45) and x['ESP'] == 1:
        return 1
    else:
        return 0
# simple_updater(g8,group_name="g8")
rf_updater(g8,group_name="g8")

In [ ]:
train_x

In [ ]:
train_x['ESP'].value_counts()

# Saving Your Model

We'd like to output the PDL to some permanent location for grading purposes. The lines below do this.

In [ ]:
import dill as pickle # you will probably need to install dill, which you do w pip install dill in your command line
with open('pdl.pkl', 'wb') as pickle_file:
    pickle.dump(f, pickle_file)

If you saved your PDL to pdl.pkl and you want to reload it, you can do so as follows (instead of re-building it from scratch every time you shut down you kernel). Just be sure that your final PDL is fully replicable in the final version of your code, so that we can re-build it just given your gs and hs.

In [ ]:
with open('pdl.pkl', 'rb') as pickle_file:
    content = pickle.load(pickle_file)

# Analysis of Your Final Model

1. How does your final model perform? On both the validation set and the training data, calculate f's error rates on each of the groups you identified, calculate the error rates of the initial model on each of the groups you identified, and compare them by taking their difference. Hint: you can use the helper function `bountyHuntWrapper.measure_group_error(model, g, x, y)` to get the error of f on x and y restricted to just the datapoints in a group g, and you can use `metrics.zero_one_loss` for the initial model (which is just a DL so you can directly use the scikit.learn functions on it).
2. Say instead you used bootstrapped fairness to postprocess equal error rates on the initial model over the groups you discovered (assuming you had a way to identify those groups ahead of time). How much would you need to inflate each groups' error to get them equal?

# Data Exploration

In order to find promising groups, you may find it helpful to do some data exploration. Please include any code or visualizations that you did to do so here. To get you started, here are some things you may find useful:

1. How to grab the predictions of the current PDL on the training data: because f.predict takes a single value as input, you have to use an apply function for this.

In [ ]:
preds = train_x.apply(f.predict, axis=1)

2. Getting the zero-one loss of a model restricted to a group you have defined.

In [ ]:
g = lambda x: 1 #here we define a group that just is all the data, replace as you see fit.
bountyHuntWrapper.measure_group_error(f, g, train_x, train_y)

3. You can view the training data by calling `train_x`. If you want to only view the data for a single group defined by your group function, you can run the following:

In [ ]:
# replace g with whatever your group is
indices = train_x.apply(g, axis=1) == 1
xs = train_x[indices]
ys = train_y[indices]

4. Inspecting the existing PDL: The PDL is stored as an object, and tracks its training errors, validation set errors, and the group functions that are used in lists where the ith element is the group errors of all groups discovered so far on the ith node in the PDL. If you are more curious about the implementation, you can look at the model.py file in the codebase, which doesn't contain anything you can use to adaptively modify your code. (But lives in the same folder as the rest of the codebase just to make importing things easier)

In [ ]:
# f is the current model
# print(f.train_errors) # group errors on training set.
# print(f.train_errors[0]) # this is the group error of each group on the initial PDL. The ith element of f.train_errors is the group error of each group on the ith version of the PDL.
# print(f.test_errors) # group errors on validation set
# print(f.predicates) # all of the group functions that have been appended so far
# print(f.leaves) # all of the h functions appended so far
# print(f.pred_names) # the names you passed in for each of the group functions, to more easily understand which are which.
for element in f.test_errors:
    print (element)

5. Looking at the group error of the ith group over each round of updates: Say you found a group at round 5 and you want to know how its group error looked at previous or subsequent rounds. To do so, you can pull `f.train_errors` or `f.test_errors` and look at the ith element of each list as follows:

In [ ]:
target_group = 0 # this sets the group whose error you want to look at at each round to the initial model. If I wanted to look at the 1st group introduced, would change to a 1, e.g.
group_errs = [f.train_errors[i][target_group] for i in range(len(f.train_errors))]
group_errs